<center> <h1> It is a copy of the 3rd practical lesson</h1>
<h1> I made modifications at the end of this notebook</h1>
</center>

In [ ]:
import numpy as np
from sklearn import preprocessing
import copy

In [ ]:
def activation(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def dactivation(x):
    return np.exp(-x)/((1+np.exp(-x))**2)
plt.plot(segedX,dactivation(segedX))

In [ ]:

class MLP:
    
    
    def __init__(self, *args):

        np.random.seed(123)

        self.shape = args
        n = len(args)

        self.layers = []

        self.layers.append(np.ones(self.shape[0]+1))

        for i in range(1,n):
            self.layers.append(np.ones(self.shape[i]))

        self.weights = []
        for i in range(n-1):
            self.weights.append(np.zeros((self.layers[i].size,
                                         self.layers[i+1].size)))

        self.dw = [0,]*len(self.weights)

        self.reset()
    

    def reset(self):
        for i in range(len(self.weights)):
 
            Z = np.random.random((self.layers[i].size,self.layers[i+1].size))

            self.weights[i][...] = (2*Z-1)*1

    
    def propagate_forward(self, data):

        self.layers[0][0:-1] = data

        for i in range(1,len(self.shape)):
            self.layers[i][...] = activation(np.dot(self.layers[i-1],self.weights[i-1]))

        return self.layers[-1]

    
    def propagate_backward(self, target, lrate=0.1):
        deltas = []
        
        error = -(target-self.layers[-1]) # y-y_kalap
        
        delta = np.multiply(error,dactivation(np.dot(self.layers[-2],self.weights[-1])))
        deltas.append(delta)
        
        for i in range(len(self.shape)-2,0,-1):
        
            delta=np.dot(deltas[0],self.weights[i].T)*dactivation(np.dot(self.layers[i-1],self.weights[i-1]))
            deltas.insert(0,delta)            
        
        for i in range(len(self.weights)):
            layer = np.atleast_2d(self.layers[i])
            delta = np.atleast_2d(deltas[i])
        
            dw = -lrate*np.dot(layer.T,delta)
        
            self.weights[i] += dw 

        
            self.dw[i] = dw

        
        return (error**2).sum()

In [ ]:
def learn(network, X, Y, valid_split, test_split, epochs=20, lrate=0.1):


        X_train = X[0:int(nb_samples*(1-valid_split-test_split))]
        Y_train = Y[0:int(nb_samples*(1-valid_split-test_split))]
        X_valid = X[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
        Y_valid = Y[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
        X_test  = X[int(nb_samples*(1-test_split)):]
        Y_test  = Y[int(nb_samples*(1-test_split)):]
    

        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_valid = scaler.transform(X_valid)
        X_test  = scaler.transform(X_test)
    

        randperm = np.random.permutation(len(X_train))
        X_train, Y_train = X_train[randperm], Y_train[randperm]
        

        for i in range(epochs):

            train_err = 0
            for k in range(X_train.shape[0]):
                network.propagate_forward( X_train[k] )
                train_err += network.propagate_backward( Y_train[k], lrate )
            train_err /= X_train.shape[0]


            valid_err = 0
            o_valid = np.zeros(X_valid.shape[0])
            for k in range(X_valid.shape[0]):
                o_valid[k] = network.propagate_forward(X_valid[k])
                valid_err += (o_valid[k]-Y_valid[k])**2
            valid_err /= X_valid.shape[0]

            print("%d epoch, train_err: %.4f, valid_err: %.4f" % (i, train_err, valid_err))


        print("\n--- TESZTELÉS ---\n")
        test_err = 0
        o_test = np.zeros(X_test.shape[0])
        for k in range(X_test.shape[0]):
            o_test[k] = network.propagate_forward(X_test[k])
            test_err += (o_test[k]-Y_test[k])**2
            print(k, X_test[k], '%.2f' % o_test[k], ' (elvart eredmeny: %.2f)' % Y_test[k])
        test_err /= X_test.shape[0]

        fig1=plt.figure()
        plt.scatter(X_test[:,0], X_test[:,1], c=np.round(o_test[:]), cmap=plt.cm.cool)

In [ ]:

network = MLP(2,10,1)

In [ ]:

nb_samples=1000
X = np.zeros((nb_samples,2))
Y = np.zeros(nb_samples)
for i in range(0,nb_samples,4):
    noise = np.random.normal(0,1,8)
    X[i], Y[i] = (-2+noise[0],-2+noise[1]), 0
    X[i+1], Y[i+1] = (2+noise[2],-2+noise[3]), 1
    X[i+2], Y[i+2] = (-2+noise[4],2+noise[5]), 1
    X[i+3], Y[i+3] = (2+noise[6],2+noise[7]), 0


fig1=plt.figure()
plt.scatter(X[:,0],X[:,1],c=Y[:], cmap=plt.cm.cool)

In [ ]:

network.reset()
learn(network, X, Y, 0.2, 0.1)

<br><br><br><br><br><br>
# HF2 modification by RZ3RVX
The previous code snippets are the original scripts for the 3rd course

Since utilizing the momentum and l1, l2 optimization methods only affect updating the weights, i am only modifying the back propagation method!


In [ ]:
# Create a new Class that implements Momentum, L1 and L2 regularization
# I am inheriting from the already implemented MLP class since i am going to override the propagate_backward function
# HF2 start momentum
class MLP_momentum(MLP):
    def propagate_backward(self, target, lrate=0.1, momentum_factor=0.75):
        deltas = []
        error = -(target-self.layers[-1])
        delta = np.multiply(error,dactivation(np.dot(self.layers[-2],self.weights[-1])))
        deltas.append(delta)
        
        # Calculating gradients in the hidden layers
        for i in range(len(self.shape)-2,0,-1):
            delta=np.dot(deltas[0],self.weights[i].T)*dactivation(np.dot(self.layers[i-1],self.weights[i-1]))
            deltas.insert(0,delta)            
        
        # --------- Interesting part:
        # Updating the weights
        for i in range(len(self.weights)):
            # Calculating the derivative of the cost function
            layer = np.atleast_2d(self.layers[i])
            delta = np.atleast_2d(deltas[i])
            cost_func_derivate = np.dot(layer.T,delta)
            
            # deltaW in time t, it depends on deltaW in time (t-1)
            deltaW_t = -lrate*cost_func_derivate + momentum_factor*self.dw[i]
            # Update weights using deltaW_t
            self.weights[i] += deltaW_t
            # Store the deltaW
            self.dw[i] = deltaW_t
        
        # Return the error
        return (error**2).sum()
# HF2 end momentum

In [ ]:
# HF2 start l1reg
class MLP_l1reg(MLP):
    def propagate_backward(self, target, lrate=0.1, lambda_1=0.0001):
        deltas = []
        error = -(target-self.layers[-1])
        delta = np.multiply(error,dactivation(np.dot(self.layers[-2],self.weights[-1])))
        deltas.append(delta)
        
        # Calculating gradients in the hidden layers
        for i in range(len(self.shape)-2,0,-1):
            delta=np.dot(deltas[0],self.weights[i].T)*dactivation(np.dot(self.layers[i-1],self.weights[i-1]))
            deltas.insert(0,delta)            
        
        # --------- Interesting part:
        # Updating the weights
        for i in range(len(self.weights)):
            # Calculating the derivative of the cost function
            layer = np.atleast_2d(self.layers[i])
            delta = np.atleast_2d(deltas[i])
            cost_func_derivate = np.dot(layer.T,delta)
            
            # For L1 reg the Weights in t+1 equals to:
            # Weights(t) - lrate*(derivative_of_weights) - lrate*lambda1*sign(weights)
            # As we discussed in the lectures...
            
            # deltaW in time t
            deltaW_t = -lrate*cost_func_derivate - lrate*lambda_1*np.sign(self.weights[i])
            # Update weights using deltaW_t
            self.weights[i] += deltaW_t
            # Store the deltaW
            self.dw[i] = deltaW_t
        
        # Return the error
        return (error**2).sum()
# HF2 end l1reg

In [ ]:
# HF2 start l2reg
class MLP_l2reg(MLP):
    def propagate_backward(self, target, lrate=0.1, lambda_2=0.001):
        deltas = []
        error = -(target-self.layers[-1])
        delta = np.multiply(error,dactivation(np.dot(self.layers[-2],self.weights[-1])))
        deltas.append(delta)
        
        # Calculating gradients in the hidden layers
        for i in range(len(self.shape)-2,0,-1):
            delta=np.dot(deltas[0],self.weights[i].T)*dactivation(np.dot(self.layers[i-1],self.weights[i-1]))
            deltas.insert(0,delta)            
        
        # --------- Interesting part:
        # Updating the weights
        for i in range(len(self.weights)):
            # Calculating the derivative of the cost function
            layer = np.atleast_2d(self.layers[i])
            delta = np.atleast_2d(deltas[i])
            cost_func_derivate = np.dot(layer.T,delta)
            
            # For L2 reg the Weights in t+1 equals to:
            # Weights(t) - lrate*(derivative_of_weights) - lrate*lambda2*weights
            # As we discussed in the lectures...
            
            # deltaW in time t
            deltaW_t = -lrate*cost_func_derivate - lrate*lambda_2*self.weights[i]
            # Update weights using deltaW_t
            self.weights[i] += deltaW_t
            # Store the deltaW
            self.dw[i] = deltaW_t
        
        # Return the error
        return (error**2).sum()
# HF2 end l2reg